# Bayesian optimization of RVCGP system

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

import tengp
import numpy as np
from gpbenchmarks import get_data
from sklearn.metrics import mean_squared_error
import random

def pdivide(x, y):
    return np.divide(x, y, out=np.copy(x), where=x!=0)

def plog(x, _):
    return np.log(x, out=np.copy(x), where=x>0)

def psin(x, _):
    return np.sin(x)

def pcos(x, _):
    return np.cos(x)

def pow2(x):
    return x**2

def pow3(x):
    return x**3


funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.subtract, 2)
funset.add(np.multiply, 2)
funset.add(pdivide, 2)

X, y = get_data('nguyenf4', 20, -1, 1)
X = np.c_[np.ones(len(X)), X]

params = tengp.Parameters(2, 1, 1, 10, funset, real_valued=True)

builder = tengp.individual.IndividualBuilder(params)
bounds = builder.create().bounds[:]

In [11]:
def cost_function(*args, **kwargs):
    G = [v for k, v in kwargs.items()]
    individual = tengp.individual.NPIndividual(G, bounds, params)
    pred = individual.transform(X)
    
    try:
        out = mean_squared_error(pred, y)
    except ValueError:
        out = np.inf
    return -out

In [12]:
from bayes_opt import BayesianOptimization

In [13]:
pbounds = {f'x{i:02}':(0, b) for i, b in enumerate(bounds)}

In [15]:
bo = BayesianOptimization(f=cost_function,
                          pbounds=pbounds)

bo.maximize(init_points=2, n_iter=25, acq="ucb", kappa=1)

Initialization
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |       x00 |       x01 |       x02 |       x03 |       x04 |       x05 |       x06 |       x07 |       x08 |       x09 |       x10 |       x11 |       x12 |       x13 |       x14 |       x15 |       x16 |       x17 |       x18 |       x19 |       x20 |       x21 |       x22 |       x23 |       x24 |       x25 |       x26 |       x27 |       x28 |       x29 |       x30 | 
    1 | 00m00s |   -4.34794 |    1.3447 |    0.5969 |    0.4007 |    1.3710 |    1.9886 |    0.7825 |    2.4164 |    0.5489 |    2.3204 |    0.3162 |    0.2972 |    0.3396 |    2.0

In [16]:
bo

In [17]:
bo = BayesianOptimization(f=cost_function,
                          pbounds=pbounds)

bo.maximize(init_points=2, n_iter=25, acq="ucb", kappa=10)

Initialization
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |       x00 |       x01 |       x02 |       x03 |       x04 |       x05 |       x06 |       x07 |       x08 |       x09 |       x10 |       x11 |       x12 |       x13 |       x14 |       x15 |       x16 |       x17 |       x18 |       x19 |       x20 |       x21 |       x22 |       x23 |       x24 |       x25 |       x26 |       x27 |       x28 |       x29 |       x30 | 
    1 | 00m00s | -180.55408 |    2.7549 |    0.6477 |    0.3589 |    2.6495 |    1.8628 |    0.9043 |    1.5895 |    2.2777 |    0.0015 |    2.6685 |    1.4171 |    1.3318 |    0.1

/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:146: RuntimeWarning: invalid value encountered in multiply
  upper_function = a*( fun_upper((1-c) * self.nodes[c_lower].value, c * self.nodes[c_upper].value))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').